In [138]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, IntegerType

spark = SparkSession.builder.appName('ToDataframe').getOrCreate()


# We read a file with minPartitions equal 3

rdd_data = spark.sparkContext.textFile('inputs/pokemon.csv',minPartitions=5).zipWithIndex()

rdd_filter = rdd_data.filter(lambda x: x[1] >= 1) \
                     .map(lambda x: x[0].split(",")) \
                     .filter(lambda x: int(x[11])==1)

rdd_get = rdd_filter.map(lambda x: ( x[2],x[1] ) )

out = rdd_get.groupByKey().mapValues(list)

print(out.take(10))

spark.stop()

[('Normal', ['Pidgey', 'Pidgeotto', 'Pidgeot', 'PidgeotMega Pidgeot', 'Rattata', 'Raticate', 'Spearow', 'Fearow', 'Jigglypuff', 'Wigglytuff', 'Meowth', 'Persian', 'Farfetchd', 'Doduo', 'Dodrio', 'Lickitung', 'Chansey', 'Kangaskhan', 'KangaskhanMega Kangaskhan', 'Tauros', 'Ditto', 'Eevee', 'Porygon', 'Snorlax']), ('Ground', ['Sandshrew', 'Sandslash', 'Diglett', 'Dugtrio', 'Cubone', 'Marowak', 'Rhyhorn', 'Rhydon']), ('Fire', ['Charmander', 'Charmeleon', 'Charizard', 'CharizardMega Charizard X', 'CharizardMega Charizard Y', 'Vulpix', 'Ninetales', 'Growlithe', 'Arcanine', 'Ponyta', 'Rapidash', 'Magmar', 'Flareon', 'Moltres']), ('Electric', ['Pikachu', 'Raichu', 'Magnemite', 'Magneton', 'Voltorb', 'Electrode', 'Electabuzz', 'Jolteon', 'Zapdos']), ('Fighting', ['Mankey', 'Primeape', 'Machop', 'Machoke', 'Machamp', 'Hitmonlee', 'Hitmonchan']), ('Rock', ['Geodude', 'Graveler', 'Golem', 'Onix', 'Omanyte', 'Omastar', 'Kabuto', 'Kabutops', 'Aerodactyl', 'AerodactylMega Aerodactyl']), ('Ghost', ['

In [139]:

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()
data = [ ("36636","Finance",3000,"USA"), 
    ("40288","Finance",5000,"IND"), 
    ("42114","Sales",3900,"USA"), 
    ("39192","Marketing",2500,"CAN"), 
    ("34534","Sales",6500,"USA") ]
schema = StructType([
     StructField('id', StringType(), True),
     StructField('dept', StringType(), True),
     StructField('salary', IntegerType(), True),
     StructField('location', StringType(), True)
     ])

df = spark.createDataFrame(data=data,schema=schema)
df.printSchema()
df.show(truncate=False)

#Convert scolumns to Map
from pyspark.sql.functions import col,lit,create_map
df = df.withColumn("propertiesMap",create_map(
        lit("salary"),col("salary"),
        lit("location"),col("location")
        )).drop("salary","location")
df.printSchema()
df.show(truncate=False)

root
 |-- id: string (nullable = true)
 |-- dept: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- location: string (nullable = true)

+-----+---------+------+--------+
|id   |dept     |salary|location|
+-----+---------+------+--------+
|36636|Finance  |3000  |USA     |
|40288|Finance  |5000  |IND     |
|42114|Sales    |3900  |USA     |
|39192|Marketing|2500  |CAN     |
|34534|Sales    |6500  |USA     |
+-----+---------+------+--------+

root
 |-- id: string (nullable = true)
 |-- dept: string (nullable = true)
 |-- propertiesMap: map (nullable = false)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

+-----+---------+---------------------------------+
|id   |dept     |propertiesMap                    |
+-----+---------+---------------------------------+
|36636|Finance  |{salary -> 3000, location -> USA}|
|40288|Finance  |{salary -> 5000, location -> IND}|
|42114|Sales    |{salary -> 3900, location -> USA}|
|39192|Marketing|{salary -> 2500, 